<a href="https://colab.research.google.com/github/KimJungMook/LLM/blob/main/LLM_Chapter2%2C3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello")

Hello


In [ ]:
input_text = "나는 최근 파리 여행을 다녀왔다"
input_text_list = input_text.split()
print("input_text_list: ", input_text_list)

input_text_list:  ['나는', '최근', '파리', '여행을', '다녀왔다']


In [ ]:
str2idx = {word: idx for idx, word in enumerate(input_text_list)}
idx2str = {idx : word for idx, word in enumerate(input_text_list)}
print("str2idx : ", str2idx)
print("idx2str : ", idx2str)

str2idx :  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
idx2str :  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}


In [ ]:
input_ids = [str2idx[word] for word in input_text_list]
print("input_ids: ", input_ids)

input_ids:  [0, 1, 2, 3, 4]


In [ ]:
import torch
import torch.nn as nn
embedding_dim = 16
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

input_embeddings = embed_layer(torch.tensor(input_ids))
input_embeddings = input_embeddings.unsqueeze(0)
print(input_embeddings.shape)

torch.Size([1, 5, 16])


In [ ]:
embedding_dim = 16
max_position = 12
embed_layer = nn.Embedding(len(str2idx), embedding_dim)
position_embed_layer = nn.Embedding(max_position, embedding_dim)

position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
position_encodings = position_embed_layer(position_ids)
token_embeddings = embed_layer(torch.tensor(input_ids))
token_embeddings = token_embeddings.unsqueeze(0)
input_embeddings = token_embeddings + position_encodings

print(input_embeddings.shape)

torch.Size([1, 5, 16])


In [ ]:
head_dim = 16

weight_q = nn.Linear(embedding_dim, head_dim)
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)

querys = weight_q(input_embeddings)
keys = weight_k(input_embeddings)
values = weight_v(input_embeddings)

In [ ]:
from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_casual=False):
  dim_k = querys.size(-1)
  scores = querys @ keys.transpose(-2,-1) / sqrt(dim_k)
  if is_casual:
    query_length = querys.size(-2)
    key_length = keys.size(-2)
    temp_mask = torch.ones(query_length, key_length, dtype=torch.bool).tril(diagonal=0)
    scores = scores.masked_fill(temp_mask == False, float("-inf"))
  weights = F.softmax(scores, dim=-1)
  return weights @ values

In [ ]:
print("원본 입력 형태: ", input_embeddings.shape)

after_attention_embeddings = compute_attention(querys, keys, values)

print("어텐션 적용 후 형태: ", after_attention_embeddings.shape)

원본 입력 형태:  torch.Size([1, 5, 16])
어텐션 적용 후 형태:  torch.Size([1, 5, 16])


In [ ]:
class AttentionHead(nn.Module):
  def __init__(self, token_embed_dim, head_dim, is_casual=False):
    super().__init__()
    self.is_casual = is_casual
    self.weight_q = nn.Linear(token_embed_dim, head_dim)
    self.weight_k = nn.Linear(token_embed_dim, head_dim)
    self.weight_v = nn.Linear(token_embed_dim, head_dim)

  def forward(self, querys, keys, values):
    outputs = compute_attention(
        self.weight_q(querys),
        self.weight_k(keys),
        self.weight_v(values),
        is_casual=self.is_casual
    )

    return outputs
attention_head = AttentionHead(embedding_dim, embedding_dim)
after_attention_embeddings = attention_head(input_embeddings, input_embeddings, input_embeddings)
print(after_attention_embeddings.shape)

torch.Size([1, 5, 16])


In [ ]:
class MultiheadAttention(nn.Module):
  def __init__(self, token_embed_dim, d_model, n_head, is_casual=False):
    super().__init__()
    self.n_head = n_head
    self.is_casual = is_casual
    self.weight_q = nn.Linear(token_embed_dim, d_model)
    self.weight_k = nn.Linear(token_embed_dim, d_model)
    self.weight_v = nn.Linear(token_embed_dim, d_model)
    self.concat_linear = nn.Linear(d_model, d_model)

  def forward(self, querys, keys, values):
    B, T, C = querys.size()
    querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1,2)
    keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1,2)
    values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1,2)
    attention = compute_attention(querys, keys, values, self.is_casual)
    output = attention.transpose(1,2).contiguous().view(B, T, C)
    output = self.concat_linear(output)
    return output

n_head = 4
mh_attention = MultiheadAttention(embedding_dim, embedding_dim, n_head)
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)
after_attention_embeddings.shape

torch.Size([1, 5, 16])

In [ ]:
norm = nn.LayerNorm(embedding_dim)
norm_x = norm(input_embeddings)
print(norm_x.shape)
print(norm_x.mean(dim=-1).data, norm_x.std(dim=-1).data)


torch.Size([1, 5, 16])
tensor([[-1.4901e-08,  0.0000e+00,  7.4506e-09, -1.4901e-08,  7.4506e-09]]) tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]])


In [ ]:
class PreLayerNorFeedForward(nn.Module):
  def __init__(self, d_model, dim_feedforward, dropout):
    super().__init__()
    self.linear1 = nn.Linear(d_model, dim_feedforward)
    self.linear2 = nn.Linear(dim_feedforward, d_model)
    self.dropout1 = nn.Dropout(dropout)
    self.dropout2 = nn.Dropout(dropout)
    self.activation = nn.GELU()
    self.norm = nn.LaterNorm(d_model)

  def forward(self, src):
    x = self.norm(src)
    x = x + self.linear2(self.dropout1(self.activation(self.linear1(x))))
    x = self.dropout2(x)
    return x

In [ ]:
class TransformerEncoderLayer(nn.Module):
  def __init__(self, d_model, nhead, dim_feedforward, dropout):
    super().__init__()
    self.attn = MultiheadAttention(d_model, d_model, nhead)
    self.norm1 = nn.LayerNorm(d_model)
    self.dropout1 = nn.Dropout(dropout)
    self.feed_forward = PreLayerNorFeedForward(d_model, dim_feedforward, dropout)

  def forward(self, src):
    norm_x = self.norm1(src)
    attn_output = self.attn(norm_x, norm_x, norm_x)
    x = src + self.dropout1(attn_output)

    x = self.feed_forward(x)
    return x


In [ ]:
import copy
def get_clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class TransformerEncoder(nn.Module):
  def __init__(self, encoder_layer, num_layers):
    super().__init__()
    self.layers = get_clones(encoder_layer, num_layers)
    self.num_layers = num_layers
    self.norm = norm

  def forward(self, src):
    output = src
    for mod in self.layers:
      output = mod(output)
    return output

In [ ]:
class TransformerDecoderLayer(nn.Module):
  def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
    super().__init__()
    self.self_attn = MultiheadAttention(d_model, d_model, n_head)
    self.multihead_attn = MultiheadAttention(d_model, d_model, nhead)
    self.feed_forward = PreLayerNorFeedForward(d_model, dim_feedforward, dropout)

    self.norm1 = nn.LayerNorm(d_model)
    self.norm2 = nn.LayerNorm(d_model)
    self.dropout1 = nn.Dropout(dropout)
    self.dropout2 = nn.Dropout(dropout)

  def forward(self, tgt, encoder_output, is_casual=True):
    x = self.norm1(tgt)
    x = x + self.droput1(self.self_attn(x, x, x, is_casual=is_casual))
    x = self.norm2(x)
    x = x + self.dropout2(self.multihead_attn(x, encoder_output, encoder_output))
    x = self.feed_forward(x)
    return x

In [ ]:
class TransformerDecoder(nn.Module):
  def __init__(self, decoder_layer, num_layers):
    super().__init__()
    self.layers = get_clones(decoder_layer, num_layers)
    self.num_layers = num_layers

  def forward(self, tgt, src):
    output = tgt
    for mod in self.layers:
      output = mod(output, src)
    return output

In [ ]:
!pip install transformers==4.50.0 datasets==3.5.0 huggingface_hub==0.29.0 -qqq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
diffusers 0.35.2 requires huggingface-hub>=0.34.0, but you have huggingface-hub 0.29.0 which is incompatible.
gradio 5.49.1 requires huggingface-hub<2.0,>=0.33.5, but you have huggingface-hub 0.29.0 which is incompatible.


In [ ]:
from transformers import AutoModel, AutoTokenizer

text = "What is Huggingface Transformers?"
#BERT 모델 사용
bert_model = AutoModel.from_pretrained("bert-base-uncased") # 모델 불러오기
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased') # 토크나이저 불러오기
encoded_input = bert_tokenizer(text, return_tensors='pt') # 입력 토큰화
bert_output = bert_model(**encoded_input) # 모델에 입력
#GPT 2모델 사용
gpt_model = AutoModel.from_pretrained('gpt2') # 모델 불러오기
gpt_tokenizer = AutoTokenizer.from_pretrained('gpt2') # 토크나이저 불러오기
encoded_input = gpt_tokenizer(text, return_tensors='pt') # 입력 토큰화
get_output = gpt_model(**encoded_input) # 모델에 입력

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import AutoModel
model_id = 'klue/roberta-base'
model = AutoModel.from_pretrained(model_id)

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoModelForSequenceClassification
model_id = 'SamLowe/roberta-base-go_emotions'
classification_model = AutoModelForSequenceClassification.from_pretrained(model_id)

In [ ]:
from transformers import AutoModelForSequenceClassification
model_id = 'klue/roberta-base'
classification_model = AutoModelForSequenceClassification.from_pretrained(model_id)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoTokenizer
model_id = 'klue/roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
tokenized = tokenizer("토크나이저는 텍스트를 토큰 단위로 나눈다")
print(tokenized)

{'input_ids': [0, 9157, 7461, 2190, 2259, 8509, 2138, 1793, 2855, 5385, 2200, 20950, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
print(tokenizer.convert_ids_to_tokens(tokenized['input_ids']))

['[CLS]', '토크', '##나이', '##저', '##는', '텍스트', '##를', '토', '##큰', '단위', '##로', '나눈다', '[SEP]']


In [ ]:
print(tokenizer.decode(tokenized['input_ids']))

[CLS] 토크나이저는 텍스트를 토큰 단위로 나눈다 [SEP]


In [ ]:
print(tokenizer.decode(tokenized['input_ids'], skip_special_tokens=True))

토크나이저는 텍스트를 토큰 단위로 나눈다


In [ ]:
tokenizer(['첫 번째 문장', '두 번째 문장'])

{'input_ids': [[0, 1656, 1141, 3135, 6265, 2], [0, 864, 1141, 3135, 6265, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [ ]:
tokenizer([['첫 번째 문장', '두 번째 문장']])

{'input_ids': [[0, 1656, 1141, 3135, 6265, 2, 864, 1141, 3135, 6265, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
first_tokenized_result = tokenizer(['첫 번째 문장', '두 번째 문장'])['input_ids']
print(tokenizer.batch_decode(first_tokenized_result))
# ['[CLS] 첫 번째 문장 [SEP]', '[CLS] 두 번째 문장 [SEP]']

second_tokenized_result = tokenizer([['첫 번째 문장', '두 번째 문장']])['input_ids']
print(tokenizer.batch_decode(second_tokenized_result))
# ['[CLS] 첫 번째 문장 [SEP] 두 번째 문장 [SEP]']

['[CLS] 첫 번째 문장 [SEP]', '[CLS] 두 번째 문장 [SEP]']
['[CLS] 첫 번째 문장 [SEP] 두 번째 문장 [SEP]']


In [ ]:
bert_tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
print(bert_tokenizer([['첫 번째 문장', '두 번째 문장']]))
# {'input_ids': [[2, 1656, 1141, 3135, 6265, 3, 864, 1141, 3135, 6265, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
roberta_tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
print(roberta_tokenizer([['첫 번째 문장', '두 번째 문장']]))
# {'input_ids': [[2, 1656, 1141, 3135, 6265, 3, 864, 1141, 3135, 6265, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
en_roberta_tokenizer = AutoTokenizer.from_pretrained('roberta-base')
print(en_roberta_tokenizer([['first sentence', 'second sentence']]))
# {'input_ids': [[0, 9502, 3645, 2, 2, 10815, 3645, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1]]}

{'input_ids': [[2, 1656, 1141, 3135, 6265, 3, 864, 1141, 3135, 6265, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
{'input_ids': [[0, 1656, 1141, 3135, 6265, 2, 864, 1141, 3135, 6265, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
{'input_ids': [[0, 9502, 3645, 2, 2, 10815, 3645, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1]]}


In [ ]:
tokenizer(['첫 번째 문장은 짧다', '두 번째 문장은 첫 번째 문장보다 더 길다.'], padding='longest')
# {'input_ids': [[0, 1656, 1141, 3135, 6265, 2073, 1599, 2062, 2, 1, 1, 1, 1, 1, 1, 1, 1], [0, 864, 1141, 3135, 6265, 2073, 1656, 1141, 3135, 6265, 2178, 2062, 831, 647, 2062, 18, 2]],
# 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
# 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

{'input_ids': [[0, 1656, 1141, 3135, 6265, 2073, 1599, 2062, 2, 1, 1, 1, 1, 1, 1, 1, 1], [0, 864, 1141, 3135, 6265, 2073, 1656, 1141, 3135, 6265, 2178, 2062, 831, 647, 2062, 18, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
from datasets import load_dataset
klue_mrc_dataset = load_dataset('klue', 'mrc')
klue_mrc_dataset

ValueError: Feature type 'List' not found. Available feature types: ['Value', 'ClassLabel', 'Translation', 'TranslationVariableLanguages', 'LargeList', 'Sequence', 'Array2D', 'Array3D', 'Array4D', 'Array5D', 'Audio', 'Image', 'Video', 'Pdf']

In [ ]:
# 로컬의 데이터 활용하는 방법

# 로컬의 csv 데이터 파일 활용
# data = load_dataset("csv", data_files="my_file.csv")

# 파이썬 딕셔너리 활용
from datasets import Dataset
my_dict = {"a" : [1,2,3]}
dataset = Dataset.from_dict(my_dict)

# 판다스 데이터 프레임 활용
import pandas as pd
df = pd.DataFrame({"a" : [1,2,3]})
dataset = Dataset.from_pandas(df)

In [ ]:
# 기사의 카테고리를 분류하는 텍스트 분류 모델 학습
klue_tc_train = load_dataset('klue', 'ynat', split='train')
klue_tc_eval = load_dataset('klue', 'ynat', split='validation')
klue_tc_train[0]

{'guid': 'ynat-v1_train_00000',
 'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영',
 'label': 3,
 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0008508947',
 'date': '2016.06.30. 오전 10:36'}

In [ ]:
klue_tc_train.features['label'].names

['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']

In [ ]:
klue_tc_train = klue_tc_train.remove_columns(['guid', 'url', 'date'])
klue_tc_eval = klue_tc_eval.remove_columns(['guid', 'url', 'date'])
klue_tc_train

# 출력 결과
# Dataset({
#     features: ['title', 'label'],
#     num_rows: 45678
# })

Dataset({
    features: ['title', 'label'],
    num_rows: 45678
})

In [ ]:
print(klue_tc_train.features['label'])
# ClassLabel(names=['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치'], id=None)

print(klue_tc_train.features['label'].int2str(1))
# 경제

klue_tc_label = klue_tc_train.features['label']

def make_str_label(batch):
  batch['label_str'] = klue_tc_label.int2str(batch['label'])
  return batch

klue_tc_train = klue_tc_train.map(make_str_label, batched=True, batch_size=1000)
klue_tc_train[0]
# {'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영', 'label': 3, 'label_str': '생활문화'

ClassLabel(names=['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치'], id=None)
경제


Map:   0%|          | 0/45678 [00:00<?, ? examples/s]

{'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영', 'label': 3, 'label_str': '생활문화'}

In [ ]:
train_dataset = klue_tc_train.train_test_split(test_size=10000, shuffle=True, seed=42)['test']
dataset = klue_tc_eval.train_test_split(test_size=1000, shuffle=True, seed=42)
test_dataset = dataset['test']
valid_dataset = dataset['train'].train_test_split(test_size=1000, shuffle=True, seed=42)['test']

In [ ]:
import torch
import numpy as np

from transformers import(
    Trainer,
    TrainingArguments,
    AutoModelForSequenceClassification,
    AutoTokenizer
)

def tokenize_function(examples):
  return tokenizer(examples["title"], padding="max_length", truncation=True)

model_id = "klue/roberta-base"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=len(train_dataset.features['label'].names))
tokenizer = AutoTokenizer.from_pretrained(model_id)
train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir = "./result",
    num_train_epochs = 1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    push_to_hub=False
)

def compute_metrics(eval_pred):
  logits , labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return {"accuracy" : (predictions == labels).mean()}

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = valid_dataset,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

trainer.train()

trainer.evaluate(test_dataset)

/tmp/ipython-input-1539903471.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.


ValueError: API key must be 40 characters long, yours was 1

In [ ]:
import torch
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from torch.optim import AdamW

def tokenize_function(examples): # 제목 컬럼에 대한 토큰화
  return tokenizer(examples["title"], padding="max_length", truncation=True)

# 모델과 토크나이저 불러오기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "klue/roberta-base"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=len(train_dataset.features['label'].names))
tokenizer = AutoTokenizer.from_pretrained(model_id)

model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
def make_dataloader(dataset, batch_size, shuffle=True):
  dataset = dataset.map(tokenize_function, batched=True).with_format("torch") # 데이터셋에 토큰화 수행
  dataset = dataset.rename_column("label", "labels") # 컬럼이름 변경
  dataset = dataset.remove_columns(column_names=['title']) # 불필요한 컬럼 제거
  return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

# 데이터 로더 만들기
train_dataloader = make_dataloader(train_dataset, batch_size=8, shuffle=True)
valid_dataloader = make_dataloader(valid_dataset, batch_size=8, shuffle=False)
test_dataloader = make_dataloader(test_dataset, batch_size=8, shuffle=False)



Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
def train_epoch(model, data_loader, optimizer):
  model.train()
  total_loss = 0
  for batch in tqdm(data_loader):
    optimizer.zero_grad()
    input_ids = batch['input_ids'].to(device)  # 모델에 입력할 토큰 아이디
    attention_mask = batch['attention_mask'].to(device) # 모델에 입력할 어텐션 마스크
    labels = batch['labels'].to(device) # 모델에 입력할 레이블
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels) # 모델 계산
    loss = outputs.loss # 손실
    loss.backward() # 역전파
    optimizer.step() # 모델 업데이트
    total_loss += loss.item()
  avg_loss = total_loss / len(data_loader)
  return avg_loss


In [ ]:
def evaluate(model, data_loader):
  model.eval()
  total_loss = 0
  predictions = []
  true_labels = []
  with torch.no_grad():
    for batch in tqdm(data_loader):
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
      logits = outputs.logits
      loss = outputs.loss
      total_loss += loss.item()
      preds = torch.argmax(logits, dim=-1)
      predictions.extend(preds.cpu().numpy())
      true_labels.extend(labels.cpu().numpy())
  avg_loss = total_loss / len(data_loader)
  accuracy = np.mean(np.asarray(predictions) == np.asarray(true_labels))
  return avg_loss, accuracy

In [ ]:
num_epochs = 1
optimizer = AdamW(model.parameters(), lr=5e-5)

# 학습 루프
for epoch in range(num_epochs):
  print(f"Epoch {epoch+1}/{num_epochs}")
  train_loss = train_epoch(model, train_dataloader, optimizer)
  print(f"Training loss : {train_loss}")
  valid_loss, valid_accuracy = evaluate(model, valid_dataloader)
  print(f"Validation loss: {valid_loss}")
  print(f"Validation accuracy: {valid_accuracy}")

_, test_accuracy = evaluate(model, test_dataloader)
print(f"Test accuracy: {test_accuracy}")

Epoch 1/1


  0%|          | 0/1250 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from huggingface_hub import login

login(token="내 토큰")
repo_id = f"jungmookkim/roberta-base-klue-ynat-classification"
# Trainier를 사용한 경우
# trainer.push_to_hub(repo_id)

# 직접 학습한 경우
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jungmookkim/roberta-base-klue-ynat-classification/commit/0c3d946dd435252d31521dd830f677354ab0eb85', commit_message='Upload tokenizer', commit_description='', oid='0c3d946dd435252d31521dd830f677354ab0eb85', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jungmookkim/roberta-base-klue-ynat-classification', endpoint='https://huggingface.co', repo_type='model', repo_id='jungmookkim/roberta-base-klue-ynat-classification'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline

model_id = "jungmookkim/roberta-base-klue-ynat-classification"

model_pipeline = pipeline("text-classification", model=model_id)

model_pipeline(dataset["test"]["title"][:5])

Device set to use cpu


[{'label': 'LABEL_6', 'score': 0.17677682638168335},
 {'label': 'LABEL_6', 'score': 0.17506270110607147},
 {'label': 'LABEL_6', 'score': 0.17837730050086975},
 {'label': 'LABEL_6', 'score': 0.17928853631019592},
 {'label': 'LABEL_6', 'score': 0.175114706158638}]

In [ ]:
import torch
from torch.nn.functional import softmax
from transformers import AutoModelForSequenceClassification, AutoTokenizer

class CustomPipeline:
  def __init__(self, model_id):
    self.model = AutoModelForSequenceClassification.from_pretrained(model_id)
    self.tokenizer = AutoTokenizer.from_pretrained(model_id)
    self.model.eval()

  def __call__(self, texts):
    tokenized = self.tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
      outputs = self.model(**tokenized)
      logits = outputs.logits

    probabilities = softmax(logits, dim=-1)
    scores, labels = torch.max(probabilities, dim=-1)
    labels_str = [self.model.config.id2label[label_idx] for label_idx in labels.tolist()]

    return [{"label" : label, "score" : score.item()} for label, score in zip(labels_str, scores)]

custom_pipeline = CustomPipeline(model_id)
custom_pipeline(dataset['train']['title'][:5])

[{'label': 'LABEL_6', 'score': 0.17567618191242218},
 {'label': 'LABEL_6', 'score': 0.17734737694263458},
 {'label': 'LABEL_6', 'score': 0.17465706169605255},
 {'label': 'LABEL_6', 'score': 0.17725373804569244},
 {'label': 'LABEL_6', 'score': 0.18183442950248718}]